In [1]:
import numpy as np
import pandas as pd 

import scipy as sp
from scipy.signal import savgol_filter

from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA

from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
init_notebook_mode(connected=True)

import os

print(np.__version__)
print(pd.__version__)
print(sp.__version__)

1.18.1
1.0.2
1.3.1


In [2]:
#select spectral area
def select_areas(df, a=0, b=0,c=0,d=0,e=0,f=0):
    area = np.r_[abs(a*2-600*2):abs(b*2-600*2+1), abs(c*2-600*2):abs(d*2-600*2+1), abs(e*2-600*2):abs(f*2-600*2+1)]
    df = df.iloc[:, area]
    return df

#apply filter
def apply_savgol(X, window_length = 25, polyorder = 2, deriv = 2):
    Xsg = savgol_filter(x = X, window_length = window_length, polyorder = polyorder, deriv = deriv)
    Xsg = pd.DataFrame(data=Xsg)
    Xsg.columns = X.columns
    Xsg.index = X.index
    return Xsg

#Show spectral plot
def show_spectra(X):
    fig = go.Figure()
    for n in range(0, len(X)):
        fig.add_trace(go.Scatter(
            x = X.columns,
            y = X.iloc[n],
            name = X.index.astype('str')[n]
            ))
    fig.show()

#Choose number of n_components based on RMSECV, RMSE
def parameter_search(parameter, y, cvkfold = 10, components = 20):
                            rmsecv = []
                            rmse = []
                            r2 = []
                            r2_adj = []
                            for n in range(1,components):
                                pls = PLSRegression(n_components=n, scale = False)
                                pls.fit(parameter,y)
                                y_cv = cross_val_predict(pls, parameter,y, cv=cvkfold)
                                y_pred = pls.predict(parameter)
                                rmsecv.append(np.sqrt(mean_squared_error(y, y_cv)))
                                rmse.append(np.sqrt(mean_squared_error(y, y_pred)))
                                r2.append(r2_score(y, y_pred))
                                r2_adj.append(1-(1-(r2_score(y, y_pred)))*(len(y)-1)/(len(y)-n-1))
                            #plot
                            fig = go.Figure()
                            fig.add_trace(go.Scatter(
                                    x = list(range(1,components)),
                                    y = r2,
                                    name = 'r2'
                                    ))
                            fig.add_trace(go.Scatter(
                                    x = list(range(1,components)),
                                    y = r2_adj,
                                    name = 'r2_adj'
                                    ))
                            fig.add_trace(go.Scatter(
                                    x = list(range(1,components)),
                                    y = rmsecv,
                                    name = 'rmsecv'
                                    ))
                            fig.add_trace(go.Scatter(
                                    x = list(range(1,components)),
                                    y = rmse,
                                    name = 'rmse'
                                    ))
                            fig.show()


#check model perfomance
def results(parameter, y, pls):
    predict = pls.predict(parameter)
    pred = []
    for n in range(0, len(predict)):
        pred.append(predict[n][0])
    results = pd.DataFrame(data=(pred, y)).transpose()
    results.index = parameter.index
    results.columns = ['predicted', 'true']
    results['residual_y_variance'] = results['true'] - results['predicted']
    results['abs_residual_y_variance'] = results['residual_y_variance'].abs()
    # define hat matrix and leverage values for Y
    yscore = pls.y_scores_
    yhat = np.dot(yscore, np.dot(
        np.linalg.inv(np.dot(yscore.T, yscore)), yscore.T
                                )
                  )
    results['yleverage'] = np.diag(yhat)
    # define hat matrix and leverage values for X
    Xvariance = pd.DataFrame(columns=parameter.columns, index=parameter.index)
    results['abs_residual_x_variance'] = np.nan
    xscore = pls.x_scores_
    xleverage = np.dot(xscore, np.dot(np.linalg.inv(np.dot(xscore.T, xscore)), xscore.T))
    results['xleverage'] = np.diag(xleverage)
    for n in range(0, len(parameter)):
        Xvariance.iloc[n] = (parameter.mean() - parameter.iloc[n])
        results['abs_residual_x_variance'][n] = np.abs(Xvariance.iloc[n].mean())

    fig2 = px.scatter(
        data_frame=results,
        x=results['true'],
        y=results['predicted'],
        trendline='ols',
        hover_name=results.index,
    )
    trendline = fig2.data[1]

    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=("True vs Predicted", "Error vs True", "Influence Y plot", 'Influence X plot'))
    # true vs predicted plot
    fig.add_trace(go.Scatter(
        x=results['true'],
        y=results['predicted'],
        mode='markers',
        hovertext=results.index,
        hovertemplate='%{hovertext}: <br>true: %{x}<br />predicted: %{y}',
        hoverlabel=dict(namelength=0)
    ),
        row=1, col=1
    )
    fig.add_trace(trendline)
    fig.update_xaxes(title_text='True', row=1, col=1)
    fig.update_yaxes(title_text='Predicted', row=1, col=1)
    # error vs true plot
    fig.add_trace(go.Scatter(
        x=results['true'],
        y=results['residual_y_variance'],
        mode='markers',
        hovertext=results.index,
        hovertemplate='%{hovertext}: <br>true: %{x}<br />residual variance: %{y}',
        hoverlabel=dict(namelength=0)
    ),
        row=1, col=2
    )
    fig.update_xaxes(title_text='True', row=1, col=2)
    fig.update_yaxes(title_text='Residual y Variance', row=1, col=2)
    # influence plot Y
    fig.add_trace(go.Scatter(
        x=results['yleverage'],
        y=results['abs_residual_y_variance'],
        mode='markers',
        hovertext=results.index,
        hovertemplate='%{hovertext}: <br>abs residual y variance: %{x}<br />y leverage: %{y}',
        hoverlabel=dict(namelength=0)
    ),
        row=2, col=1
    )
    fig.update_xaxes(title_text='y Leverage', row=2, col=1)
    fig.update_yaxes(title_text='ABS Residual y Variance', row=2, col=1)
    # influence plot X
    fig.add_trace(go.Scatter(
        x=results['xleverage'],
        y=results['abs_residual_x_variance'],
        mode='markers',
        hovertext=results.index,
        hovertemplate='%{hovertext}: <br>abs residual x variance : %{x}<br />x leverage: %{y}',
        hoverlabel=dict(namelength=0)
    ),
        row=2, col=2
    )
    fig.update_xaxes(title_text='x Leverage', row=2, col=2)
    fig.update_yaxes(title_text='ABS Residual x Variance', row=2, col=2)

    fig.update_layout(showlegend=False, height=800)
    fig.show()

#wavelength pls coefficients
def show_coef(parameter, pls):
    fig = px.line(
        x = np.array(parameter.columns),
        y = pls.coef_[:,0]
    )
    fig.show()

# read files and compile them to single csv
Y_data = pd.read_csv('C:/Users/DPsoctrade/Desktop/Projects/IR_speculations/SVR_IR/exported_30-505-0043.csv', skiprows = [i for i in range(1,178)])
Y_data = Y_data.drop(columns = 'density')
X_spec = pd.read_csv('C:/Users/DPsoctrade/Desktop/Projects/IR_speculations/SVR_IR/convertedspec.csv', sep = ';')
X_spec = X_spec.drop(columns = ['sample id / wavelng', 'Unnamed: 6802'])
df = pd.DataFrame.join(Y_data, X_spec)
pd.DataFrame.to_csv(df, 'data.csv')

In [ ]:
data_path = 'C:/Users/DPSoctrade/Desktop/Projects/Grabner/0043/data_0043_13March2020'
df_all = pd.read_csv((os.path.join(data_path, 'data.csv')), dtype = {'Sample name': 'str', 'Sample type': 'str'}, index_col = 'Sample name')
df = df_all[df_all['RON[ ][Gasoline]'].notnull()]

In [ ]:
#clean data
df = df[(df['Group'] == 'Russia') | (df['Group'] == 'Paveltsevo') | (df['Group'] == 'AromaticRus')]
X = df.loc[:,'600':]
y = df['RON[ ][Gasoline]']

In [ ]:
#apply filter to data
X_sg = apply_savgol(X, window_length = 25, polyorder = 2, deriv = 2)
# Define spectral areas from 650-924 & 1360-1413
X_sg = select_areas(X_sg, a = 650, b = 924)
X_sg.head()

In [ ]:
show_spectra(X_sg)

In [ ]:
parameter_search(parameter = X_sg, y = y, cvkfold = 5, components = 20)

In [ ]:
components = 10
pls = PLSRegression(n_components=components, scale = True) 
pls.fit(X_sg,y)

In [ ]:
results(parameter = X_sg, y = y, pls = pls)

In [ ]:
pca = PCA(components)
pca.fit_transform(X_sg)
print (pca.explained_variance_ratio_.cumsum())

In [ ]:
scores = pd.DataFrame(pls.x_scores_)
PCA1 = scores[0]
PCA2 = scores[1]
fig = px.scatter(x=PCA1, y=PCA2)
fig.show()

In [ ]:
show_coef(X_sg, pls = pls)